<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier 
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
nltk.download('stopwords')
RNS = 54321

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Загрузим данные 

In [2]:
df = pd.read_csv("/datasets/toxic_comments.csv")
df.info()
df1=df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Таргет в стобце toxic

Текст в столбце text

Очистим текст и проведем лематезацию

In [4]:
documents = []
stemmer = WordNetLemmatizer()

for sen in range(0, len(df['text'])):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(df['text'][sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [5]:
df['lemm_text'] = documents

In [ ]:
# #КОД РЕВЬЮЕРА
# from tqdm import tqdm
# tqdm.pandas()
# data['text'] = data['text'].progress_apply(lemmatize_text)

Проверим данные 

In [6]:
df.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour m seemin...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man m really not trying to edit war it jus...
3,"""\nMore\nI can't make any real suggestions on ...",0,more can make any real suggestion on improveme...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


Удалим стопслова

In [7]:
stopwords = set(nltk_stopwords.words('english'))

In [8]:
X = df['lemm_text']
y = df['toxic']

Разделим на обучающую и тестовые выборки

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RNS, stratify=y) 

Проведем векторизацию текста

In [10]:
tfidfconverter = TfidfVectorizer(stop_words=stopwords) 
X_train = tfidfconverter.fit_transform(X_train)
X_test = tfidfconverter.transform(X_test)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(127656, 157015)
(31915, 157015)
(127656,)
(31915,)


## Обучение

Обучим несколько моделей, перед этим подберем гипперпараметры

### RandomForestClassifier

In [12]:
model_RF = RandomForestClassifier(random_state=RNS,  max_depth=12, n_estimators=1000, class_weight='balanced') 

In [13]:
model_RF.fit(X_train, y_train) 

RandomForestClassifier(class_weight='balanced', max_depth=12, n_estimators=1000,
                       random_state=54321)

In [14]:
y_pred = model_RF.predict(X_test)

In [15]:
f1_model_RF = f1_score(y_test, y_pred)
print(f1_score(y_test, y_pred))

0.37163063784360817



    
Так же есть и другие методы подбора гиперпараметров Optuna/Halving GridSearch:
    
+  https://medium.com/databulls/tps-mar21-leaderboard-14-xgb-catboost-lgbm-optuna-cdffb5124368
+  https://towardsdatascience.com/faster-hyperparameter-tuning-with-scikit-learn-71aa76d06f12
+  https://scikit-learn.ru/3-2-tuning-the-hyper-parameters-of-an-estimator/ 
    



### LogisticRegression

In [16]:
model_LR = LogisticRegression(random_state=RNS, C=10 , class_weight='balanced', max_iter=1000) 

In [17]:
model_LR.fit(X_train, y_train) 

LogisticRegression(C=10, class_weight='balanced', max_iter=1000,
                   random_state=54321)

In [18]:
y_pred = model_LR.predict(X_test)

In [19]:
f1_model_LR = f1_score(y_test, y_pred)
print(f1_score(y_test, y_pred))

0.7641550208903616


### LGBMClassifier

In [20]:
model_LGC = LGBMClassifier(random_state=RNS, max_depth=9, num_leaves=60)

In [21]:
model_LGC.fit(X_train, y_train) 

LGBMClassifier(max_depth=9, num_leaves=60, random_state=54321)

In [22]:
y_pred = model_LGC.predict(X_test)

In [23]:
f1_model_LGC = f1_score(y_test, y_pred)
print(f1_score(y_test, y_pred))

0.6904901386447959


### CatBoost

Для модели  CatBoostClassifier ограничим длину вектора в 1500

(и уменьшим выборку)

In [24]:
sample_size = 5000
df = df.sample(n=sample_size,random_state=RNS).reset_index(drop=True)
X = df['lemm_text']
y = df['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RNS, stratify=y) 
tfidfconverter = TfidfVectorizer(stop_words=stopwords, max_features=1500)
X_train = tfidfconverter.fit_transform(X_train)
X_test = tfidfconverter.transform(X_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4000, 1500)
(1000, 1500)
(4000,)
(1000,)


In [25]:
model_CBR = CatBoostClassifier(random_state=RNS, depth=10, iterations=1000, learning_rate=0.1)

In [26]:
model_CBR.fit(X_train, y_train) 

0:	learn: 0.6011651	total: 1.75s	remaining: 29m 11s
1:	learn: 0.5375869	total: 3.35s	remaining: 27m 49s
2:	learn: 0.4734488	total: 5.05s	remaining: 28m
3:	learn: 0.4258121	total: 6.76s	remaining: 28m 2s
4:	learn: 0.3920432	total: 8.46s	remaining: 28m 3s
5:	learn: 0.3640811	total: 10.3s	remaining: 28m 18s
6:	learn: 0.3432929	total: 11.9s	remaining: 28m 12s
7:	learn: 0.3237568	total: 13.6s	remaining: 28m 2s
8:	learn: 0.3068992	total: 15.1s	remaining: 27m 43s
9:	learn: 0.2935051	total: 16.8s	remaining: 27m 44s
10:	learn: 0.2831053	total: 18.5s	remaining: 27m 45s
11:	learn: 0.2735761	total: 20.1s	remaining: 27m 38s
12:	learn: 0.2634449	total: 21.9s	remaining: 27m 43s
13:	learn: 0.2548890	total: 23.5s	remaining: 27m 35s
14:	learn: 0.2489533	total: 25.1s	remaining: 27m 29s
15:	learn: 0.2428538	total: 26.8s	remaining: 27m 29s
16:	learn: 0.2385977	total: 28.4s	remaining: 27m 22s
17:	learn: 0.2351063	total: 30.2s	remaining: 27m 29s
18:	learn: 0.2304962	total: 31.9s	remaining: 27m 27s
19:	learn:

In [27]:
y_pred = model_CBR.predict(X_test)

In [28]:
f1_model_CBR = f1_score(y_test, y_pred)
print(f1_score(y_test, y_pred))

0.6153846153846154


 Tfidf можно не использовать для CatBoost(внутри него есть встроенные механизмы для работы с текстом):


+  https://habr.com/ru/post/555064/

</div>

## Выводы

In [29]:
result = pd.DataFrame([f1_model_RF,f1_model_LR, f1_model_LGC, f1_model_CBR], 
                      index = ['RandomForestClassifier', 'LogisticRegression', 'LGBMClassifier', 'CatBoostClassifier'], 
                      columns = ['f1_test'])
result

,f1_test
RandomForestClassifier,0.371631
LogisticRegression,0.764155
LGBMClassifier,0.690490
CatBoostClassifier,0.615385


Лучший результат со значением метрики F1 больше  0.75 у модели LogisticRegression 
Модель, обученная не на всех данных показывает значение метрики F1 меньше 0.75, а также в случае ограничения длины вектора на 1500 



Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множество реализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 
    
</div>

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны